# <center>Detroit Lions Offensive Line Analysis</center>

<img src='https://static.clubs.nfl.com/image/upload/t_editorial_landscape_12_desktop/lions/p3q8ujlelurs5m2uu1kt' style='width:800px;margin:auto'/>

In [68]:
#import libraries and datasets
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

games = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/games.csv')
players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/players.csv')
player_play = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/player_play.csv')
plays = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/plays.csv')
week4 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/tracking_week_4.csv')

/kaggle/input/nfl-big-data-bowl-2025/players.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_7.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_9.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_6.csv
/kaggle/input/nfl-big-data-bowl-2025/games.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_8.csv
/kaggle/input/nfl-big-data-bowl-2025/player_play.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_4.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_3.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_5.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_1.csv
/kaggle/input/nfl-big-data-bowl-2025/plays.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_2.csv


## <center>Overview</center>

The main objective of this notebook is to analyze how the offensive linemen can help us predict whether the play is going to be a run or a pass play. We will need to overlap this with formation of the offense to see more valuable insights (an empty formation is almost certainly a pass play)

In [69]:
det_sea = week4[week4['gameId'] == 2022100205]
det_sea_plays = plays[plays['gameId'] == 2022100205]
det_sea[det_sea['club'] == 'DET']['displayName'].unique()

array(['Michael Brockers', 'Charles Harris', 'Alex Anzalone',
       'DeShon Elliott', 'Amani Oruwariye', 'Jeff Okudah', 'Alim McNeill',
       'Derrick Barnes', 'Aidan Hutchinson', 'Kerby Joseph',
       'Malcolm Rodriguez', 'Mike Hughes', 'Isaiah Buggs',
       'Julian Okwara', 'Benito Jones', 'Chris Board', 'Jared Goff',
       'Taylor Decker', 'Josh Reynolds', 'Jamaal Williams', 'Dan Skipper',
       'Frank Ragnow', 'Evan Brown', 'T.J. Hockenson', 'Tom Kennedy',
       'Quintez Cephus', 'Penei Sewell', 'Kalif Raymond',
       'Justin Jackson', 'Brock Wright', 'Matt Nelson', 'Craig Reynolds',
       'Austin Bryant', 'Will Harris', 'James Mitchell',
       'Maurice Alexander', 'Kayode Awosika'], dtype=object)

In [70]:
pd.set_option('display.max_colwidth', None)
detroit_plays = det_sea_plays[det_sea_plays['possessionTeam'] == 'DET']
detroit_plays[detroit_plays['passResult'].isna()]['offenseFormation'].value_counts()

detroit_plays[detroit_plays['offenseFormation'] == 'SINGLEBACK']

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
5,2022100205,2314,(14:15) Ja.Williams up the middle to DET 32 for 3 yards (C.Barton; U.Nwosu).,3,2,6,DET,SEA,DET,29,...,3,-0.024210,0.024210,-0.442517,False,MAN,NaN,0,Cover 6-Left,Zone
201,2022100205,2625,(7:11) M.Nelson reported in as eligible. Ja.Williams right end to DET 49 for 3 yards (S.Jones; B.Mafe).,3,1,10,DET,SEA,DET,46,...,3,-0.012892,0.012892,-0.324285,False,MAN,NaN,0,Cover-1,Man
762,2022100205,1468,(9:02) J.Goff pass short left to Ja.Williams pushed ob at DET 17 for 1 yard (B.Mafe).,2,1,10,DET,SEA,DET,16,...,1,-0.022358,0.022358,-0.600837,True,OUTSIDE ZONE,NaN,0,Cover-3,Zone
1607,2022100205,1084,(15:00) C.Reynolds up the middle pushed ob at SEA 27 for 21 yards (T.Woolen).,2,2,10,DET,SEA,SEA,48,...,21,0.043709,-0.043709,1.705531,False,POWER,NaN,0,Quarters,Zone
2280,2022100205,1915,(2:56) J.Goff pass short left to Q.Cephus to SEA 46 for 9 yards (J.Brooks).,2,1,10,DET,SEA,DET,45,...,9,0.020759,-0.020759,0.813811,True,NaN,NaN,0,Cover-2,Zone
3210,2022100205,1133,(14:00) Ja.Williams right tackle to SEA 31 for 1 yard (J.Brooks; U.Nwosu).,2,1,15,DET,SEA,SEA,32,...,1,-0.010306,0.010306,-0.476627,False,COUNTER,NaN,0,Cover-6 Right,Zone
3350,2022100205,2377,"(12:34) J.Goff pass short right to J.Jackson to DET 38 for 2 yards (B.Mafe, C.Barton).",3,2,10,DET,SEA,DET,36,...,2,-0.025951,0.025951,-0.862718,True,NaN,NaN,0,Cover-3,Zone
5115,2022100205,2648,"(6:39) Ja.Williams up the middle for 51 yards, TOUCHDOWN.",3,2,7,DET,SEA,DET,49,...,51,0.063580,-0.063580,4.549468,False,POWER,NaN,0,Cover-2,Zone
6248,2022100205,443,"(8:16) Ja.Williams left tackle to DET 31 for 5 yards (Q.Diggs, C.Bryant).",1,1,10,DET,SEA,DET,26,...,5,0.001581,-0.001581,-0.036492,False,OUTSIDE ZONE,NaN,0,Quarters,Zone
6524,2022100205,1497,(8:32) Ja.Williams left end to DET 21 for 4 yards (J.Brooks).,2,2,9,DET,SEA,DET,17,...,4,-0.005084,0.005084,-0.210817,False,POWER,NaN,0,Cover-3,Zone


In [71]:
detroit_plays[detroit_plays['offenseFormation'] == 'SINGLEBACK']
run_sb = week4[(week4['playId'] == 2648) & (week4['gameId'] == 2022100205)]
pass_sb = week4[(week4['playId'] == 1570) & (week4['gameId'] == 2022100205)]

### <center>Run play</center>

In [72]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'



fig_dict = {
    'data': [],
    'layout': {},
    'frames': []
}

fig_dict['layout']['xaxis'] = {
    'range': [0, 100],
    'title': {'text': 'x', 'font': {'color': 'white'}},  
    'linecolor': 'black',  
    'tickcolor': 'black',  
    'dtick': 10, 
    'showgrid': True 
}

fig_dict['layout']['yaxis'] = {
    'range': [0, 50],
    'title': {'text': 'y', 'font': {'color': 'white'}},
    'linecolor': 'black', 
    'tickcolor': 'black',  
    'dtick': 10, 
    'showgrid': False  
}

fig_dict['layout']['hovermode'] = 'closest'
fig_dict['layout']['font'] = {'color': 'white'} 
fig_dict['layout']['updatemenus'] = [
    {
        "buttons": [
            {
                'args': [None, {'frame': {'duration': 50, 'redraw': False},
                                'fromcurrent': True, 'transition': {'duration': 1000}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False},
                                  'mode': 'immediate',
                                  'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]


sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20, 'color': 'black'},
        'prefix': 'Time:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300},
    'pad': {'b': 10, 't': 50},
    'len': .9,
    'x': .1,
    'y': 0,
    'steps': []
}

for club in pass_sb['club'].unique():
    data_dict = {
        'x': pass_sb.loc[pass_sb['club'] == club, 'x'],
        'y': pass_sb.loc[pass_sb['club'] == club, 'y'],
        'mode': 'markers',
        'text': pass_sb.loc[pass_sb['club'] == club, 'displayName'],
        'name': club
    }
    fig_dict['data'].append(data_dict)

initial_frame = {'data': [], 'name': 'Initial'}
fig_dict['frames'].append(initial_frame)

for frame in pass_sb['frameId'].unique():
    f = {'data': [], 'name': str(frame)}
    
    for club in pass_sb['club'].unique():
        frame_data = {
            'x': pass_sb.loc[pass_sb['frameId'] == frame, 'x'][pass_sb['club'] == club],
            'y': pass_sb.loc[pass_sb['frameId'] == frame, 'y'][pass_sb['club'] == club],
            'mode': 'markers',
            'text': pass_sb.loc[pass_sb['frameId'] == frame, 'displayName'][pass_sb['club'] == club],
            'name': club
        }
        f['data'].append(frame_data) 

    fig_dict['frames'].append(f)
    
    slider_step = {
        'args': [
            [frame],
            {'frame': {'duration': 0, 'redraw': False},
             'mode': 'immediate',
             'transition': {'duration': 0}}
        ],
        'label': str(frame),
        'method': 'animate'
    }
    sliders_dict['steps'].append(slider_step) 

fig_dict['layout']['sliders'] = [sliders_dict]

fig = go.Figure(fig_dict)

if len(fig_dict['frames']) > 1:
    fig.update(data=fig_dict['frames'][1]['data']) 

fig.update_layout(paper_bgcolor='#242424', plot_bgcolor='#0a4200')
    
fig.show()

### <center>Pass play</center>

In [73]:
fig_dict = {
    'data': [],
    'layout': {},
    'frames': []
}

fig_dict['layout']['xaxis'] = {
    'range': [0, 100],
    'title': {'text': 'x', 'font': {'color': 'white'}},  
    'linecolor': 'black',  
    'tickcolor': 'black',  
    'dtick': 10, 
    'showgrid': True 
}

fig_dict['layout']['yaxis'] = {
    'range': [0, 50],
    'title': {'text': 'y', 'font': {'color': 'white'}},
    'linecolor': 'black', 
    'tickcolor': 'black',  
    'dtick': 10, 
    'showgrid': False  
}

fig_dict['layout']['hovermode'] = 'closest'
fig_dict['layout']['font'] = {'color': 'white'} 
fig_dict['layout']['updatemenus'] = [
    {
        "buttons": [
            {
                'args': [None, {'frame': {'duration': 50, 'redraw': False},
                                'fromcurrent': True, 'transition': {'duration': 1000}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False},
                                  'mode': 'immediate',
                                  'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]


sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20, 'color': 'black'},
        'prefix': 'Time:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300},
    'pad': {'b': 10, 't': 50},
    'len': .9,
    'x': .1,
    'y': 0,
    'steps': []
}

for club in pass_sb['club'].unique():
    data_dict = {
        'x': pass_sb.loc[pass_sb['club'] == club, 'x'],
        'y': pass_sb.loc[pass_sb['club'] == club, 'y'],
        'mode': 'markers',
        'text': pass_sb.loc[pass_sb['club'] == club, 'displayName'],
        'name': club
    }
    fig_dict['data'].append(data_dict)

initial_frame = {'data': [], 'name': 'Initial'}
fig_dict['frames'].append(initial_frame)

for frame in pass_sb['frameId'].unique():
    f = {'data': [], 'name': str(frame)}
    
    for club in pass_sb['club'].unique():
        frame_data = {
            'x': pass_sb.loc[pass_sb['frameId'] == frame, 'x'][pass_sb['club'] == club],
            'y': pass_sb.loc[pass_sb['frameId'] == frame, 'y'][pass_sb['club'] == club],
            'mode': 'markers',
            'text': pass_sb.loc[pass_sb['frameId'] == frame, 'displayName'][pass_sb['club'] == club],
            'name': club
        }
        f['data'].append(frame_data) 

    fig_dict['frames'].append(f)
    
    slider_step = {
        'args': [
            [frame],
            {'frame': {'duration': 0, 'redraw': False},
             'mode': 'immediate',
             'transition': {'duration': 0}}
        ],
        'label': str(frame),
        'method': 'animate'
    }
    sliders_dict['steps'].append(slider_step) 

fig_dict['layout']['sliders'] = [sliders_dict]

fig = go.Figure(fig_dict)

if len(fig_dict['frames']) > 1:
    fig.update(data=fig_dict['frames'][1]['data']) 

fig.update_layout(paper_bgcolor='#242424', plot_bgcolor='#0a4200')
    
fig.show()

In [74]:
run_lineset = pd.DataFrame()

for player in run_sb['nflId'].unique().tolist()[:-1]:
    df_tobeadded = run_sb[run_sb['nflId'] == player]
    lineset_index = df_tobeadded.index[df_tobeadded['event'] == 'line_set'].tolist()
    lineset_index = lineset_index[0]
    ballsnap_index = df_tobeadded.index[df_tobeadded["event"] == "ball_snap"].tolist()
    ballsnap_index = ballsnap_index[0]

    df_tobeadded = df_tobeadded.loc[lineset_index:ballsnap_index]

    run_lineset = pd.concat([run_lineset, df_tobeadded])

ol_players = ['Penei Sewell', 'Evan Brown', 'Frank Ragnow', 'Dan Skipper', 'Taylor Decker']

ol = run_lineset[run_lineset['displayName'].isin(ol_players)]
o_avg_run = ol.groupby('displayName')['o'].mean().reset_index()
o_avg_run.rename(columns={'o': 'avg_o'}, inplace=True)
o_avg_run

,displayName,avg_o
0,Dan Skipper,86.090000
1,Evan Brown,107.011818
2,Frank Ragnow,89.655455
3,Penei Sewell,95.530909
4,Taylor Decker,90.299091


In [75]:
pass_lineset = pd.DataFrame()

for player in pass_sb['nflId'].unique().tolist()[:-1]:
    df_tobeadded = pass_sb[pass_sb['nflId'] == player]
    lineset_index = df_tobeadded.index[df_tobeadded['event'] == 'line_set'].tolist()
    lineset_index = lineset_index[0]
    ballsnap_index = df_tobeadded.index[df_tobeadded["event"] == "ball_snap"].tolist()
    ballsnap_index = ballsnap_index[0]

    df_tobeadded = df_tobeadded.loc[lineset_index:ballsnap_index]

    pass_lineset = pd.concat([pass_lineset, df_tobeadded])

ol = pass_lineset[pass_lineset['displayName'].isin(ol_players)]
o_avg_pass = ol.groupby('displayName')['o'].mean().reset_index()
o_avg_pass.rename(columns={'o': 'avg_o'}, inplace=True)
o_avg_pass

,displayName,avg_o
0,Dan Skipper,94.939206
1,Evan Brown,78.167619
2,Frank Ragnow,82.293810
3,Penei Sewell,82.354921
4,Taylor Decker,70.731429


### <center>Now we need to expound this for the whole using data for singleback formation</center>

In [76]:
sb_runs = detroit_plays[(detroit_plays['offenseFormation'] == 'SINGLEBACK') & (detroit_plays['passResult'].isna())]
sb_passes = detroit_plays[(detroit_plays['offenseFormation'] == 'SINGLEBACK') & (detroit_plays['passResult'].notna())]

sb_runs_playids = sb_runs['playId'].tolist()
sb_passes_playids = sb_passes['playId'].tolist()

runs_raw = pd.DataFrame()
runs = pd.DataFrame()

for play in sb_runs_playids:
    tobeadded = week4[(week4['gameId'] == 2022100205) & (week4['playId'] == play)]
    players_add = [x for x in tobeadded['nflId'].unique().tolist() if x == x]
    
    for player in players_add:
        tba = tobeadded[tobeadded['nflId'] == player]
        lineset_index = tba.index[tba['event'] == 'line_set'].tolist()
        lineset_index = lineset_index[0]
        ballsnap_index = tba.index[tba["event"] == "ball_snap"].tolist()
        ballsnap_index = ballsnap_index[0]

        tba = tba.loc[lineset_index:ballsnap_index]
        runs = pd.concat([runs, tba])

runs[runs['club'] == 'DET']

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
4307252,2022100205,2314,43290.0,Jared Goff,46,BEFORE_SNAP,2022-10-02 18:49:09.7,16.0,DET,right,37.15,26.85,0.11,0.13,0.01,83.70,293.22,line_set
4307253,2022100205,2314,43290.0,Jared Goff,47,BEFORE_SNAP,2022-10-02 18:49:09.8,16.0,DET,right,37.15,26.85,0.09,0.09,0.01,83.70,300.29,NaN
4307254,2022100205,2314,43290.0,Jared Goff,48,BEFORE_SNAP,2022-10-02 18:49:09.9,16.0,DET,right,37.15,26.83,0.06,0.07,0.02,83.70,264.58,NaN
4307255,2022100205,2314,43290.0,Jared Goff,49,BEFORE_SNAP,2022-10-02 18:49:10,16.0,DET,right,37.14,26.85,0.06,0.05,0.01,83.70,304.35,NaN
4307256,2022100205,2314,43290.0,Jared Goff,50,BEFORE_SNAP,2022-10-02 18:49:10.1,16.0,DET,right,37.16,26.84,0.02,0.03,0.02,85.24,347.01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4290064,2022100205,1939,53436.0,Penei Sewell,106,BEFORE_SNAP,2022-10-02 18:19:45.9,58.0,DET,right,62.83,23.56,0.00,0.00,0.00,85.66,187.63,NaN
4290065,2022100205,1939,53436.0,Penei Sewell,107,BEFORE_SNAP,2022-10-02 18:19:46,58.0,DET,right,62.83,23.56,0.00,0.00,0.00,85.66,211.17,NaN
4290066,2022100205,1939,53436.0,Penei Sewell,108,BEFORE_SNAP,2022-10-02 18:19:46.1,58.0,DET,right,62.83,23.56,0.02,0.28,0.00,85.66,227.45,NaN
4290067,2022100205,1939,53436.0,Penei Sewell,109,BEFORE_SNAP,2022-10-02 18:19:46.2,58.0,DET,right,62.82,23.55,0.08,0.63,0.01,86.88,217.94,NaN


In [77]:
passes_raw = pd.DataFrame()
passes = pd.DataFrame()

for play in sb_passes_playids:
    tobeadded = week4[(week4['gameId'] == 2022100205) & (week4['playId'] == play)]
    players_add = [x for x in tobeadded['nflId'].unique().tolist() if x == x]
    
    for player in players_add:
        tba = tobeadded[tobeadded['nflId'] == player]
        lineset_index = tba.index[tba['event'] == 'line_set'].tolist()
        lineset_index = lineset_index[0]
        ballsnap_index = tba.index[tba["event"] == "ball_snap"].tolist()
        ballsnap_index = ballsnap_index[0]

        tba = tba.loc[lineset_index:ballsnap_index]
        passes = pd.concat([passes, tba])
        
passes[passes['playDirection'] == 'right']

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
4249291,2022100205,1468,35562.0,Al Woods,42,BEFORE_SNAP,2022-10-02 18:01:17,99.0,SEA,right,27.08,29.86,0.08,0.05,0.01,256.60,199.18,line_set
4249292,2022100205,1468,35562.0,Al Woods,43,BEFORE_SNAP,2022-10-02 18:01:17.1,99.0,SEA,right,27.09,29.86,0.05,0.03,0.01,255.77,177.62,NaN
4249293,2022100205,1468,35562.0,Al Woods,44,BEFORE_SNAP,2022-10-02 18:01:17.2,99.0,SEA,right,27.09,29.87,0.04,0.03,0.01,255.77,183.37,NaN
4249294,2022100205,1468,35562.0,Al Woods,45,BEFORE_SNAP,2022-10-02 18:01:17.3,99.0,SEA,right,27.08,29.88,0.03,0.02,0.01,255.77,196.09,NaN
4249295,2022100205,1468,35562.0,Al Woods,46,BEFORE_SNAP,2022-10-02 18:01:17.4,99.0,SEA,right,27.08,29.90,0.01,0.02,0.02,256.70,275.27,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268640,2022100205,1570,54618.0,Tariq Woolen,85,BEFORE_SNAP,2022-10-02 18:03:47.4,27.0,SEA,right,50.31,36.61,0.42,0.41,0.04,258.51,53.76,NaN
4268641,2022100205,1570,54618.0,Tariq Woolen,86,BEFORE_SNAP,2022-10-02 18:03:47.5,27.0,SEA,right,50.35,36.64,0.45,0.34,0.04,257.92,49.06,NaN
4268642,2022100205,1570,54618.0,Tariq Woolen,87,BEFORE_SNAP,2022-10-02 18:03:47.6,27.0,SEA,right,50.38,36.67,0.48,0.27,0.05,256.91,46.37,NaN
4268643,2022100205,1570,54618.0,Tariq Woolen,88,BEFORE_SNAP,2022-10-02 18:03:47.7,27.0,SEA,right,50.42,36.71,0.52,0.24,0.05,256.91,44.45,NaN


In [78]:
passes.loc[passes['playDirection'] == 'left', 'x'] = passes.loc[passes['playDirection'] == 'left', 'x'].apply(lambda x: 100 - x)
passes.loc[passes['playDirection'] == 'left', 'y'] = passes.loc[passes['playDirection'] == 'left', 'y'].apply(lambda y: 50 - y)
passes.loc[passes['playDirection'] == 'left', 'o'] = passes.loc[passes['playDirection'] == 'left', 'o'].apply(lambda o: 360 - o)

runs.loc[runs['playDirection'] == 'left', 'x'] = runs.loc[runs['playDirection'] == 'left', 'x'].apply(lambda x: 100 - x)
runs.loc[runs['playDirection'] == 'left', 'y'] = runs.loc[runs['playDirection'] == 'left', 'y'].apply(lambda y: 50 - y)
runs.loc[runs['playDirection'] == 'left', 'o'] = runs.loc[runs['playDirection'] == 'left', 'o'].apply(lambda o: 360 - o)


runs[(runs['playDirection'] == 'right') & (runs['club'] == 'DET')]

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
4307252,2022100205,2314,43290.0,Jared Goff,46,BEFORE_SNAP,2022-10-02 18:49:09.7,16.0,DET,right,37.15,26.85,0.11,0.13,0.01,83.70,293.22,line_set
4307253,2022100205,2314,43290.0,Jared Goff,47,BEFORE_SNAP,2022-10-02 18:49:09.8,16.0,DET,right,37.15,26.85,0.09,0.09,0.01,83.70,300.29,NaN
4307254,2022100205,2314,43290.0,Jared Goff,48,BEFORE_SNAP,2022-10-02 18:49:09.9,16.0,DET,right,37.15,26.83,0.06,0.07,0.02,83.70,264.58,NaN
4307255,2022100205,2314,43290.0,Jared Goff,49,BEFORE_SNAP,2022-10-02 18:49:10,16.0,DET,right,37.14,26.85,0.06,0.05,0.01,83.70,304.35,NaN
4307256,2022100205,2314,43290.0,Jared Goff,50,BEFORE_SNAP,2022-10-02 18:49:10.1,16.0,DET,right,37.16,26.84,0.02,0.03,0.02,85.24,347.01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4290064,2022100205,1939,53436.0,Penei Sewell,106,BEFORE_SNAP,2022-10-02 18:19:45.9,58.0,DET,right,62.83,23.56,0.00,0.00,0.00,85.66,187.63,NaN
4290065,2022100205,1939,53436.0,Penei Sewell,107,BEFORE_SNAP,2022-10-02 18:19:46,58.0,DET,right,62.83,23.56,0.00,0.00,0.00,85.66,211.17,NaN
4290066,2022100205,1939,53436.0,Penei Sewell,108,BEFORE_SNAP,2022-10-02 18:19:46.1,58.0,DET,right,62.83,23.56,0.02,0.28,0.00,85.66,227.45,NaN
4290067,2022100205,1939,53436.0,Penei Sewell,109,BEFORE_SNAP,2022-10-02 18:19:46.2,58.0,DET,right,62.82,23.55,0.08,0.63,0.01,86.88,217.94,NaN


In [79]:
runs[(runs['playDirection'] == 'left') & (runs['club'] == 'DET')]

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
4152268,2022100205,443,43290.0,Jared Goff,61,BEFORE_SNAP,2022-10-02 17:16:33.4,16.0,DET,left,14.11,20.11,0.06,0.02,0.00,110.65,45.38,line_set
4152269,2022100205,443,43290.0,Jared Goff,62,BEFORE_SNAP,2022-10-02 17:16:33.5,16.0,DET,left,14.10,20.11,0.06,0.03,0.01,110.65,46.49,NaN
4152270,2022100205,443,43290.0,Jared Goff,63,BEFORE_SNAP,2022-10-02 17:16:33.6,16.0,DET,left,14.09,20.10,0.06,0.03,0.01,109.83,47.21,NaN
4152271,2022100205,443,43290.0,Jared Goff,64,BEFORE_SNAP,2022-10-02 17:16:33.7,16.0,DET,left,14.10,20.09,0.06,0.03,0.01,109.83,33.69,NaN
4152272,2022100205,443,43290.0,Jared Goff,65,BEFORE_SNAP,2022-10-02 17:16:33.8,16.0,DET,left,14.10,20.09,0.04,0.03,0.01,108.63,24.70,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4154999,2022100205,443,53436.0,Penei Sewell,120,BEFORE_SNAP,2022-10-02 17:16:39.3,58.0,DET,left,14.30,17.06,0.00,0.06,0.00,88.12,222.64,NaN
4155000,2022100205,443,53436.0,Penei Sewell,121,BEFORE_SNAP,2022-10-02 17:16:39.4,58.0,DET,left,14.30,17.06,0.04,0.44,0.00,89.12,200.17,NaN
4155001,2022100205,443,53436.0,Penei Sewell,122,BEFORE_SNAP,2022-10-02 17:16:39.5,58.0,DET,left,14.30,17.07,0.14,0.91,0.01,88.21,202.49,NaN
4155002,2022100205,443,53436.0,Penei Sewell,123,BEFORE_SNAP,2022-10-02 17:16:39.6,58.0,DET,left,14.33,17.12,0.44,1.59,0.05,89.49,206.37,NaN


In [80]:
fig_dict = {
    'data': [],
    'layout': {},
    'frames': []
}

fig_dict['layout']['xaxis'] = {
    'range': [0, 100],
    'title': {'text': 'x', 'font': {'color': 'white'}},  
    'linecolor': 'black',  
    'tickcolor': 'black',  
    'dtick': 10, 
    'showgrid': True 
}

fig_dict['layout']['yaxis'] = {
    'range': [0, 50],
    'title': {'text': 'y', 'font': {'color': 'white'}},
    'linecolor': 'black', 
    'tickcolor': 'black',  
    'dtick': 10, 
    'showgrid': False  
}

fig_dict['layout']['hovermode'] = 'closest'
fig_dict['layout']['font'] = {'color': 'white'} 
fig_dict['layout']['updatemenus'] = [
    {
        "buttons": [
            {
                'args': [None, {'frame': {'duration': 50, 'redraw': False},
                                'fromcurrent': True, 'transition': {'duration': 1000}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False},
                                  'mode': 'immediate',
                                  'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]


sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20, 'color': 'black'},
        'prefix': 'Time:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300},
    'pad': {'b': 10, 't': 50},
    'len': .9,
    'x': .1,
    'y': 0,
    'steps': []
}

for club in passes['club'].unique():
    data_dict = {
        'x': passes.loc[passes['club'] == club, 'x'],
        'y': passes.loc[passes['club'] == club, 'y'],
        'mode': 'markers',
        'text': passes.loc[passes['club'] == club, 'displayName'],
        'name': club
    }
    fig_dict['data'].append(data_dict)

initial_frame = {'data': [], 'name': 'Initial'}
fig_dict['frames'].append(initial_frame)

for frame in passes['frameId'].unique():
    f = {'data': [], 'name': str(frame)}
    
    for club in passes['club'].unique():
        frame_data = {
            'x': passes.loc[passes['frameId'] == frame, 'x'][passes['club'] == club],
            'y': passes.loc[passes['frameId'] == frame, 'y'][passes['club'] == club],
            'mode': 'markers',
            'text': passes.loc[passes['frameId'] == frame, 'displayName'][passes['club'] == club],
            'name': club
        }
        f['data'].append(frame_data) 

    fig_dict['frames'].append(f)
    
    slider_step = {
        'args': [
            [frame],
            {'frame': {'duration': 0, 'redraw': False},
             'mode': 'immediate',
             'transition': {'duration': 0}}
        ],
        'label': str(frame),
        'method': 'animate'
    }
    sliders_dict['steps'].append(slider_step) 

fig_dict['layout']['sliders'] = [sliders_dict]

fig = go.Figure(fig_dict)

if len(fig_dict['frames']) > 1:
    fig.update(data=fig_dict['frames'][1]['data']) 

fig.update_layout(paper_bgcolor='#242424', plot_bgcolor='#0a4200')
    
fig.show()

In [81]:
ol_players = ['Penei Sewell', 'Evan Brown', 'Frank Ragnow', 'Dan Skipper', 'Taylor Decker']

ol = runs[runs['displayName'].isin(ol_players)]
o_avg_run = ol.groupby('displayName')['o'].mean().reset_index()
o_avg_run.rename(columns={'o': 'avg_o'}, inplace=True)
o_avg_run

,displayName,avg_o
0,Dan Skipper,93.518414
1,Evan Brown,85.234759
2,Frank Ragnow,84.091052
3,Penei Sewell,90.578538
4,Taylor Decker,87.814670


In [82]:
ol_players = ['Penei Sewell', 'Evan Brown', 'Frank Ragnow', 'Dan Skipper', 'Taylor Decker']

ol = passes[passes['displayName'].isin(ol_players)]
o_avg_pass = ol.groupby('displayName')['o'].mean().reset_index()
o_avg_pass.rename(columns={'o': 'avg_o'}, inplace=True)
o_avg_pass

,displayName,avg_o
0,Dan Skipper,93.226060
1,Evan Brown,89.355478
2,Frank Ragnow,85.516886
3,Penei Sewell,85.954597
4,Taylor Decker,94.805385


### <center>Here we do see a slight difference, all of the pass plays are actually more oriented forward vs run plays (this could signify that it depends where the rush is planned to be headed towards)</center>

In [83]:
passes = passes.merge(players[['nflId', 'position']], on='nflId', how='left')

passes

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event,position
0,2022100205,1468,35562.0,Al Woods,42,BEFORE_SNAP,2022-10-02 18:01:17,99.0,SEA,right,27.08,29.86,0.08,0.05,0.01,256.60,199.18,line_set,NT
1,2022100205,1468,35562.0,Al Woods,43,BEFORE_SNAP,2022-10-02 18:01:17.1,99.0,SEA,right,27.09,29.86,0.05,0.03,0.01,255.77,177.62,NaN,NT
2,2022100205,1468,35562.0,Al Woods,44,BEFORE_SNAP,2022-10-02 18:01:17.2,99.0,SEA,right,27.09,29.87,0.04,0.03,0.01,255.77,183.37,NaN,NT
3,2022100205,1468,35562.0,Al Woods,45,BEFORE_SNAP,2022-10-02 18:01:17.3,99.0,SEA,right,27.08,29.88,0.03,0.02,0.01,255.77,196.09,NaN,NT
4,2022100205,1468,35562.0,Al Woods,46,BEFORE_SNAP,2022-10-02 18:01:17.4,99.0,SEA,right,27.08,29.90,0.01,0.02,0.02,256.70,275.27,NaN,NT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11721,2022100205,3227,54618.0,Tariq Woolen,84,BEFORE_SNAP,2022-10-02 19:33:43.5,27.0,SEA,left,89.42,35.29,0.01,0.01,0.01,289.28,75.47,NaN,CB
11722,2022100205,3227,54618.0,Tariq Woolen,85,BEFORE_SNAP,2022-10-02 19:33:43.6,27.0,SEA,left,89.42,35.29,0.00,0.01,0.01,289.28,80.07,NaN,CB
11723,2022100205,3227,54618.0,Tariq Woolen,86,BEFORE_SNAP,2022-10-02 19:33:43.7,27.0,SEA,left,89.42,35.31,0.00,0.01,0.02,289.28,91.94,NaN,CB
11724,2022100205,3227,54618.0,Tariq Woolen,87,BEFORE_SNAP,2022-10-02 19:33:43.8,27.0,SEA,left,89.42,35.33,0.03,0.77,0.02,287.84,182.62,NaN,CB
